# 透過目標字找到相關聯的關聯詞

## amod_word

In [ ]:
from stanfordcorenlp import StanfordCoreNLP
from nltk.tree import Tree
import json
import pandas as pd
import re
import csv


In [ ]:
#read data(review)
review_2013 = pd.read_csv("review_2013.csv", encoding = "Iso-8859-1")
print(review_2013.shape)

In [ ]:
#取出text和business_id欄位
review_text = review_2013['text']
review_businessid = review_2013['business_id']

In [ ]:
#load nlp 
nlp = StanfordCoreNLP('stanford-corenlp-full-2018-10-05')

In [ ]:
#find all business_id and export to total_id.csv
total_id = tip_df_businessid.iloc[0:]
total_id = pd.DataFrame(total_id)
total_id.to_csv('total_id.csv', encoding='utf-8')

In [ ]:
#利用 token_list中的字去找出 dependency_parse_list中的數字為哪個字
def dependency_parse_list_new_making(dpl,dplnew):
    for tup in dpl:
        tup_to_list = list(tup)
        tup_to_list[1] = token_list[tup_to_list[1] - 1]
        tup_to_list[2] = token_list[tup_to_list[2] - 1]
        #append the new tupple to dplnew
        dplnew.append(tup_to_list)
    return dplnew

In [ ]:
#create a new dict
b_id_dict = {}

for i in range(len(review_text)):
    review = review_text[i]
    business_id = review_businessid[i]
    # use ! or ? or . to split sentence
    split_sentence = re.split('!|\?|\.', review)
    for sentence in split_sentence:
        # tokenize and dependency_parse
        token_list = nlp.word_tokenize(sentence)
        dependency_parse_list = nlp.dependency_parse(sentence)
        # create new list
        dependency_parse_list_new = []
        dependency_parse_list_new = dependency_parse_list_new_making(dependency_parse_list, dependency_parse_list_new)
        # 如果business_id沒有在b_id_dict中，加進去(作為key)。並且根據business_id加入對應的value 
        if business_id not in b_id_dict.keys():
            b_id_dict[business_id] = []
            for lst in dependency_parse_list_new:
                b_id_dict[business_id].append(lst)
        else:
            for lst in dependency_parse_list_new:
                b_id_dict[business_id].append(lst)

In [ ]:
#找到與target_word_list相關聯的關聯詞(amod)，做一個只有關聯詞的list
only_relation = []
for key, value  in b_id_dict.items():
    target_word_list = ['atmosphere','happy','fun','friendly','lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food','order','ordered','staff','service','wait','server','servered','quality','price','prices','place','places','location','area']
    for targetword in target_word_list:
        for i in range(len(value)):
            if value[i][0] == 'amod' and value[i][1] == targetword:
                only_relation.append(value[i][2])
                #print(key, value[i])

In [ ]:
#list to dataframe, and export to csv
#drop duplicates
only_relation_df = pd.DataFrame(only_relation, columns = ['relationword']).drop_duplicates()
only_relation_df.to_csv('only_relaitonword.csv', encoding = 'utf-8', index = False)

In [ ]:
# 做一個有business_id、目標字和關聯詞的表
relation_key = []
relation_targetword = []
relation_word = []
for key, value in b_id_dict.items():
    target_word_list = ['atmosphere','happy','fun','friendly','lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food','order','ordered','staff','service','wait','server','servered','quality','price','prices','place','places','location','area']
    for targetword in target_word_list:
        for i in range(len(value)):
            if value[i][0] == 'amod' and value[i][1] == targetword:
                relation_key.append(key)
                relation_targetword.append(value[i][1])
                relation_word.append(value[i][2])


In [ ]:
#relation_key(list) to dataframe
df_relation_key = pd.DataFrame(relation_key, columns=['business_id'])

In [ ]:
#relaiton_targetword(list) to dataframe
df_relation_targetword = pd.DataFrame(relation_targetword, columns=['targetword'])

In [ ]:
#relation_word(list) to dataframe
df_relation_word = pd.DataFrame(relation_word, columns=['relationword'])

In [ ]:
#合併
relation_df = pd.concat([df_relation_key, df_relation_targetword, df_relation_word], axis = 1)

In [ ]:
#export to csv
relation_df.to_csv('relation_word.csv', encoding='utf-8')

---

## neg_word

### neg_word 對應到的relationword與amod相同

In [ ]:
#找到與target_word_list相關聯的關聯詞(neg)
amod_target = []
amod_relation = []

for key,value in b_id_dict.items():
    target_word_list=['atmosphere','happy','fun','friendly','lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food','order','ordered','staff','service','wait','server','servered','quality','price','prices','place','places','location','area']
    for targetword in target_word_list:
        for i in range(len(value)):
            #if value[i][0] == 'neg'and (value[i][1]== targetword or value[i][1]== targetword):
            if value[i][0] == 'amod'and value[i][1] == targetword:
                amod_target.append(value[i][1])
                amod_relation.append(value[i][2])
                #print(key, value[i])
len(amod_target)

In [ ]:
# list to dataframe
df_amod_target = pd.DataFrame(amod_target, columns=['targetword'])
df_amod_relation = pd.DataFrame(amod_relation, columns=['relationword'])
#concat
df_amod = pd.concat([df_amod_target, df_amod_relation], axis=1)

In [ ]:
#找出不重複的關聯詞
amod_list = list(set(amod_relation))

In [ ]:
#把amod的關聯詞當作neg的目標字
amod_neg_target = []
amod_neg_relation = []
for key,value in b_id_dict.items():
    for targetword in amod_list:
        for i in range(len(value)):
            if value[i][0] == 'neg'and value[i][1] == targetword:
                amod_neg_target.append(value[i][1])
                amod_neg_relation.append(value[i][2])
                print(key, value[i])

In [ ]:
#list to dataframe
amod_neg_target_df = pd.DataFrame(amod_neg_target, columns=['relationword'])
amod_neg_relation_df =  pd.DataFrame(amod_neg_relation, columns=['relationword_neg'])
#repalce and concat
amod_neg_relation_df = amod_neg_relation_df.replace("n't","not")
df_amod_neg = pd.concat([amod_neg_target_df, amod_neg_relation_df], axis=1)
len(amod_neg_relation_df)

In [ ]:
#做一個有business_id、目標字和關聯詞的表
amod_relation_key = []
amod_relation_targetword = []
amod_relation_word = []
for key, value in b_id_dict.items():
    target_word_list=['atmosphere','happy','fun','friendly','lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food','order','ordered','staff','service','wait','server','servered','quality','price','prices','place','places','location','area']
    for targetword in target_word_list: 
        for i in range(len(value)):
            if value[i][0] == 'amod' and value[i][1] == targetword:
                amod_relation_key.append(key)
                amod_relation_targetword.append(value[i][1])
                amod_relation_word.append(value[i][2])
                print(key, value)

In [ ]:
#把amod的關聯詞當作neg的目標字
relation_key = []
relation_targetword = []
relation_word = []
for key, value in b_id_dict.items():
    for targetword in amod_list:
        for i in range(len(value)):
            if value[i][0] == 'neg' and value[i][1] == targetword:
                relation_key.append(key)
                relation_targetword.append(value[i][1])
                relation_word.append(value[i][2])
                print(key,value[i])

In [ ]:
#list to dataframe
df_relation_key = pd.DataFrame(relation_key, columns=['business_id'])
df_relation_targetword = pd.DataFrame(relation_targetword, columns=['targetword'])
df_relation_word = pd.DataFrame(relation_word, columns=['relationword'])

In [ ]:
#concat and replace
relation_df = pd.concat([df_relation_key, df_relation_targetword, df_relation_word], axis = 1)
relation_df = relation_df.replace("n't", "not")


In [ ]:
#export to csv
relation_df.to_csv('relation_neg_word.csv', encoding='utf-8')

In [ ]:
nlp.close()